In [ ]:
%reload_ext autoreload
%autoreload 2

In [1]:
from typing import ClassVar
from dataclasses import dataclass

import sqlite3
import json

import cv2
import torch
import torch.nn.functional as F
from torch import nn
import numpy as np

from sensor_msgs.msg import PointCloud2
from sensor_msgs.msg import Image as ROS_Image
import sensor_msgs_py.point_cloud2 as pc2
from rclpy.serialization import deserialize_message

import matplotlib.pyplot as plt

from PIL import Image
from cv_bridge import CvBridge

from ultralytics import YOLO

from torchvision.models import resnet18 as resnet, ResNet18_Weights
from torchvision import transforms

from sklearn.linear_model import Ridge
from queue import Queue

from src.follow_me.follow_me.tracker import Tracker


In [2]:
ds = "v1"

conn = sqlite3.connect(f"./{ds}/{ds}_0.db3")
cursor = conn.cursor()

cursor.execute("SELECT * FROM topics")
topic_info = cursor.fetchall()
for topic in topic_info:
    print(topic[1])

/rosout
/events/write_split
/diagnostics
/parameter_events
/velodyne_points
/joint_states
/scan
/tf_static
/velodyne_packets
/camera/camera/color/camera_info
/camera/camera/extrinsics/depth_to_color
/camera/camera/depth/metadata
/camera/camera/depth/camera_info
/camera/camera/depth/image_rect_raw
/camera/camera/color/metadata
/camera/camera/color/image_raw
/odom
/robot_description
/tf
/tracer_status


In [3]:
topic_name = "/camera/camera/color/image_raw"
cursor.execute(f"SELECT * FROM topics WHERE name='{topic_name}'")
topic_info = cursor.fetchone()
	
if not topic_info:
	print("Topic not found.")
	
print("Topic Info:", topic_info)
topic_id = topic_info[0]  # 'id' is the first column in topics

# Step 2: Get all messages for that topic
cursor.execute("SELECT * FROM messages WHERE topic_id = ?", (topic_id,))
messages = cursor.fetchall()

Topic Info: (16, '/camera/camera/color/image_raw', 'sensor_msgs/msg/Image', 'cdr', '- history: 3\n  depth: 0\n  reliability: 1\n  durability: 1\n  deadline:\n    sec: 9223372036\n    nsec: 854775807\n  lifespan:\n    sec: 9223372036\n    nsec: 854775807\n  liveliness: 1\n  liveliness_lease_duration:\n    sec: 9223372036\n    nsec: 854775807\n  avoid_ros_namespace_conventions: false')


In [4]:
model = YOLO("yolo11n-pose.pt")
model.eval()
print()

In [6]:
bridge = CvBridge()
tracker = Tracker((256, 448))

# for idx in range(1950, 1951):
# for idx in range(1700, 1701):
for idx in range(700, 701):
    msg_id, topic_id, timestamp, data = messages[idx]
    rgb_image = deserialize_message(data, ROS_Image)
    cv_image = bridge.imgmsg_to_cv2(rgb_image, desired_encoding='bgr8')

    results = model(
        cv_image,
        conf=0.4,
        verbose=False,
    )
    result = results[0].cpu()
    
    bboxes = result.boxes.xywh.numpy()
    kpts = result.keypoints.xy.numpy()
    confs = result.keypoints.conf.numpy()
    
    all_features, visible_part = tracker.process_crop(
        cv_image, bboxes, kpts, confs
    )
    # print(all_features.shape, visible_part.shape)

    result.show()


AttributeError: 'NoneType' object has no attribute 'numpy'

In [ ]:
bboxes.shape